In [1]:
import pandas as pd 
import numpy as np 
import sqlite3 as sql
from datetime import datetime, timedelta

# Plotly
import plotly.graph_objects as go
from FX_data.subplots import * # scipt with TA charts 

# Web app
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table

database_name = "FX_data/database.db"

conn = sql.connect('FX_data/database.db')
cur = conn.cursor()
df = pd.read_sql_query("SELECT * FROM eda_data_LR ",conn)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index(['Date'],inplace=True)
conn.close()

# FOR DASH APP
cluster_options = [{'label': i, 'value': i} for i in list(df["Cluster"].unique())]
year_options = pd.DatetimeIndex(df.reset_index()['Date']).year #.year
action_options = [{'label': k, 'value': v} for k,v in {('Sell Signal',0),('Buy Signal',1)}]#[{'label': i, 'value': i} for i in list(df["Prediction"].unique())]
forecast_options = [{'label': k, 'value': v} for k,v in {('Forecasted Wrongly',0),('Forecasted Correctly',1)}]#[{'label': i, 'value': i} for i in list(df["Correct"].unique())]
dataset_options = [{'label':'All Trades','value':'eda_data_LR'}, {'label':'Profitable Subgroups','value':'profitable_subgroups'}]
model_options = [{'label': k, 'value': v} for k,v in {('Logistic Regression',0),('Random Forest',1)}],

#df = pd.read_sql_query("SELECT * FROM profitable_subgroups", conn)
df.head(1)

,Target,Prediction,Correct,% Buy,% Sell,Price,Open,High,Low,Close,Prob Buy,Prob Sell,Balance,Profit,comp_0,comp_1,t-SNE_1,t-SNE_2,Cluster,Confidence %
Date,,,,,,,,,,,,,,,,,,,,
2004-03-01,0,1,0,37.25,62.75,0.8034,0.755824,0.762779,0.758515,0.76933,0.364423,0.635577,9903.846154,-96.153846,-0.019401,0.017532,22.393435,-12.548299,5,62.75


### Layout
- [Learn More link](https://github.com/plotly/dash-sample-apps/blob/master/apps/dash-oil-and-gas/app.py)
- [features Medium example](https://medium.com/@leclercqz.acacio/build-a-dashboard-from-scratch-using-dash-python-framework-7bd0ea986f6)
- [Price charts](https://plot.ly/python/time-series/)
- [Candle sticks](https://plot.ly/python/candlestick-charts/)
##### ISSUES 
- ~~<font color=red> Setting the heigh/width on scatterplot callback interferes with layout </font>~~
    - added __ in left box to increase height 

In [2]:
# DEFINE HTML TEMPLATE
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)


app.layout = html.Div(
    [
        dcc.Store(
            id="aggregate_data", 
        ),
        # TITLE BANNER
        html.Div(
            [
                html.Div(
                    [
                        html.H1(
                        " USD/EUR TRADES ",
                        style={"margin-bottom": "0px",
                               #'textAlign': 'center',
                              },
                        ),
                        html.H3(
                        "Identifying under/over performing trades for improved profitability",
                        #" Model Performance Evaluation ",
                        style={"margin-bottom": "25px",
                               #'textAlign': 'center',
                              },
                        #style = {"line-height": "60px"},
                        ),
                    html.Hr(),
                    ],
                    className="ten columns",
                ),
                html.Div(
                    [
                        html.Img(
                            src="https://media-exp1.licdn.com/dms/image/C4D03AQEcxZXx15lL-w/profile-displayphoto-shrink_100_100/0?e=1586995200&v=beta&t=dO0XD0N3rBbGGhCBO2Ds-OZJjAvEPzpwlXZz2ps2LJI",
                            #"https://s3-us-west-1.amazonaws.com/plotly-tutorials/logo/new-branding/dash-logo-by-plotly-stripe.png",
                            style={
#                                 "height": "60px",
#                                 "width": "auto",
                                "margin-right": "0px",
                            },
                        ),
                    #html.Hr(),
                    ],className="two columns",                       
                ),           
                #
            ],
            id="header",
            className = "row flex-display",
            style={"margin-bottom": "25px"},
        ),       
        # USER OPTION PANNEL SEPARATE
        html.Div(       
            [
                html.Div(
                    [
                        html.B(
                            "User Option Panel",
                            style={
                                'textAlign': 'center',
                            },           
                        ), # name for section (Card)
                        
                        html.Hr(),  # underline for section 
                        #1 Model ( All trades / Profitable Subgroup)   <------------------ NOT MEANT TO WORK
                        html.P(
                            "Select Forecasting Model:",
                            className="control_label",
                        ),
                        dcc.Dropdown(
                            id='model_1',
                            options= model_options,
                            value= 0,
                            #style={"display": "block","margin-left": "0","width": "40%"},
                            placeholder= "Logistic Regression", 
                            searchable=False,
                            clearable=False,
                            className="dcc_control",
                        ),  
                        html.Hr(),    
                        
                        #2 SELECT Z AXIS ['Profit','Confidence %']
                        html.P(
                            "Select All Clusters / Profitable Clusters:",
                            className="control_label",
                        ),
                        dcc.Dropdown(
                            id='axis_z',
                            options= [{'label': i, 'value': i} for i in ['Profit','Confidence %']],
                            value= 'Profit',
                            #style={"display": "block","margin-left": "0","width": "40%"},
                            placeholder= "Select a axis feature", 
                            searchable=False,
                            clearable=False,
                            className="dcc_control",
                        ),  
                        html.Hr(),                        
                        
                        #3 Dataset ( All trades / Profitable Subgroup)
                        html.P(
                            "Select All Clusters / Profitable Clusters:",
                            className="control_label",
                        ),
                        dcc.Dropdown(
                            id='dataset_1',
                            options= dataset_options,
                            value= 'eda_data_LR',
                            #style={"display": "block","margin-left": "0","width": "40%"},
                            placeholder= "Select a dataset", 
                            searchable=False,
                            clearable=False,
                            className="dcc_control",
                        ),  
                        html.Hr(),
                        
                        #4 Cluster Dropdown 
                        html.P(
                            "Cluster Dropdown:",
                            className="control_panel",
                        ), 
                        dcc.Dropdown(
                            id='cluster_1',
                            options=cluster_options, 
                            multi=True,
                            value= list(range(0,10)), #list(df["Cluster"].unique().sort_values()),
                            #style={"display": "block","margin-left": "0","width": "70%"},
                            className="dcc_control",                      
                        ),  
                        #html.Hr(),
                        
                        #5 Trade Actions - Buy/Sell
                        html.P(
                            "Select Trade Type:",
                            className="control_panel",
                        ), 
                        dcc.Checklist(
                            id='action_1',
                            options=action_options,
                            value= [0,1],                            
                            style={"display": "block","margin-left": "0","width": "30%"},     
                            className="dcc_control",  
                        ),
                        html.Hr(),
                        
                        #6 Forecasts - Correct/Wrong
                        html.P(
                            "Filter Correct/False Forecasts:",
                            className="control_panel",
                        ), 
                        dcc.Checklist(
                            id='forecast_1',
                            options=forecast_options,
                            value= [0,1],                            
                            #style={"display": "block","margin-left": "0","width": "30%"},     
                            className="dcc_control",  
                        ),                        
                        #html.Hr(),
                        
                        #7 Data range slider - Years
                        html.P("Select Year:",
                            className="control_panel",
                        ),                     
                        dcc.RangeSlider( 
                            id = "date_1",
                            updatemode = "drag", 
                            min = year_options.min(),
                            max = year_options.max(),
                            marks = {i: '{}'.format(i) for i in year_options.unique().tolist()},
                            value= [2005,2006],        
                            className="dcc_control", 
                        ), 
                        html.Hr(),
                        html.Div(
                            id='slider_selection',
#                             style={"display": "block","margin-left": "auto","width": "30%"},
                        ), 
                        
                    ],
                    id= "cross-filter-options",     #'slider_selection',
                    #style={'height': '100%', 'display': 'inline-block'},
                    className="pretty_container four columns",
                    #className="four columns",

                ),
                # SCATTERPLOT SEPARATE
                html.Div(
                    [    
                        html.Div(
                            [
                                html.B(
                                    "Analyze Trader Subgroups",
                                    style={
                                        'textAlign': 'center',
                                    },   
                                ),
                                html.Hr(),
                                dcc.Graph(
                                    id = "scatter_3d",  
                                    #clickData = '2005-10-10', #  #df.index[100], 
                                    clickData = {'points': [{'customdata': '2005-03-01'}]},
                                ),
                            ],
                            id = "countGraphContainer",
                            #className="pretty_container",
                        ),
                    ],
                    id="rightCol",
                    #style={'width': '70%', 'display': 'inline-block'},
                    className="pretty_container eight columns",
                ),    
            ],
            #className="row",
            className = "row flex-display",
            #style={"margin-bottom": "25px"},
        ),
        
        # DYNAMIC SUBPLOTS FOR SCATTER-PLOT 3D
        html.Div(
            [
                # Header
                html.Div(
                    [
                        html.Hr(),
                        html.H3(
                            "TECHNICAL ANALYSIS", #TECHNICAL ANALYSIS OF PRICE DATA
                            style = {
                                     "margin-top": "0px",
                                     "margin-bottom": "0px",
                                     'textAlign': 'center',
                                    },
                            #style={"max-width": "100%", "font-size": "1.5rem", "padding": "1px 1px"},
                        ), 
                        html.H5("In depth analysis of price movements",
                            style = { 
                                     "margin-bottom": "25px",
                                     'textAlign': 'center',
                                    },
                        ),
                        html.Hr(),
                    ],
                ),
                
                # Subplot 1
                html.Div(
                    [
                        dcc.Graph(
                            id='subplot_1',    
                            style = { 
                                     "margin-left": "0",
                                     "margin-bottom": "25px",
                                     },
                            #style = { "margin-bottom": "25px","width": "70%"}, #  <--- Adjusts graph size
                        ), 
                    ],className="pretty_container five columns", 
                ),
                # Subplot 2                
                html.Div(
                    [
                        dcc.Graph(
                            id='subplot_2',  
                            style = { 
                                     "margin-right": "0",
                                     "margin-bottom": "25px",
                                     },
                        ), 
                    ],className="pretty_container six columns", 
                ),  
                # Subplot 3
                html.Div(
                    [
                        dcc.Graph(
                            id='subplot_3',  
                            style = { 
                                     "margin-left": "0",
                                     "margin-bottom": "25px",
                                     },
                        ), 
                    ],className="pretty_container five columns", 
                ),    
                # Subplot 4
                html.Div(
                    [
                        dcc.Graph(
                            id='subplot_4',   
                            style = { 
                                     "margin-right": "0",
                                     "margin-bottom": "25px",
                                     },
                        ), 
                    ],className="pretty_container six columns", 
                ),                  
  
            ],
            className="row",                
        ),  
    ],
    style={'backgroundColor':'rgb(240, 240, 240)'},
)   # END 


### Callbacks
- <font color=red> The price chart does not update dynamically, received None data from scatter_trader plot </font>
- dcc.Graph component has four attributes that can change through user-interaction: <b>hoverData, clickData, selectedData, relayoutData. </b>

- [Custom Hover Data](https://community.plot.ly/t/how-to-access-customdata-with-hover-action/26324/6)
- Example 2

In [3]:
# ============================= 3D Scatterplot =============================== #
@app.callback(
                [
                    Output('scatter_3d', 'figure'),
                    Output('slider_selection', 'children'), 
                    #Output('price_chart', 'figure'),
                  ],
               #Output('table_1', 'data'),  ], # Dynamic data table 
                [
                    Input ('model_1','value'),
                    Input ('dataset_1','value'),
                    Input ('axis_z','value'),
                    Input('cluster_1','value'),
                    Input('action_1','value'), 
                    Input('forecast_1','value'),
                    Input('date_1','value'),           
                ]   
            ) 

def plots (modelD,datasetD,axisD, clusterD, actionD, forecastD, rangeD ):  # just my names (According to order in callback)
    
    # Call data from SQL db
    conn = sql.connect(database_name)
    cur = conn.cursor()
    df = pd.read_sql_query("SELECT * FROM {} ".format(datasetD), conn)  #FROM eda_data_LR
    df["Date_Y"] = pd.DatetimeIndex(df["Date"]).year
    conn.close()    
    
    
    dataset = df[(df["Date_Y"] >= rangeD[0]) &  # Slice data according to lower year range 
                   (df["Date_Y"] <= rangeD[1]) & # Slice data according to upper year range
                   (df["Cluster"].isin(clusterD)) & # Slice data according to quarter                       
                   (df["Prediction"].isin(actionD)) &
                   (df["Correct"].isin(forecastD))# Slice data according to sector    
                  ]  
    
    trace1 = go.Scatter3d(x = dataset["t-SNE_1"],  
                          y = dataset["t-SNE_2"],
                          z = dataset[axisD], 
                          #dataset["Profit"], 
                          mode = "markers+text",
                          customdata = dataset["Date"] ,  # REQUITED FOR SUBPLOT AND HOVER ACTION!

                          # WORKS - <br> NEW LINE PER FEATURE
                          hovertext= \
                              [
                              
                               " <br>Date: {},\
                                 <br>Cluster: {},\
                                 <br>Buy Conviction: {}%,\
                                 <br>Sell Conviction: {}%,\
                                 <br>Close: {},\
                                 <br>Open: {},\
                                 <br>High: {},\
                                 <br>Low: {}, " 
                              .format(str(d)[:11],
                                           c,
                                           b,
                                           s,
                                           round(cl,2),
                                           round(cl,2),
                                           round(hi,2),
                                           round(lo,2))
                               for d,c,b,s,cl,op,hi,lo in dataset[['Date','Cluster','% Buy','% Sell','Price','Open','High','Low']].values.tolist()],
                          hoverinfo="text", 
                          
                         
                          marker = dict(size= df["Confidence %"]/4, 
                                        sizemin = 1.5,  # min size
                                        color= dataset["Cluster"],  # THIS CAN BE DIFFERENT
                                        colorscale = "Portland",
                                        colorbar = {"thickness": 10, "len": 0.5, "x": 0.8, "y": 0.6, },
                                        symbol = list(np.where(dataset['Correct'] == 1, 'diamond', 'cross')), #square #-open
                                        showscale = False,
                                       ),  
                         )
    data_1 = [ trace1 ]    
    layout_1 = go.Layout( 
             title = "<b>Scatterplot Trades</b>",
             #clickmode = 'event+select',
             # UX
             height = 850,  #1500,
             #width = 850,  #1500,    <----- Screws with layout 
             paper_bgcolor = "rgb(230, 230, 230)", # background colour "rgb(220, 235, 247)"
             # paper_bgcolor = "rgb(158, 194, 217)", # background colour 
             scene = dict( # UI
                          aspectmode = "cube",  
                          hovermode = "closest",
                          dragmode = "turntable", #  "orbit" | "turntable" | "zoom" | "pan" | False
                          xaxis = dict(title ="t-SNE_1" ),
                          yaxis = dict(title ="t-SNE_2" ),
                          zaxis = dict(title =axisD),  #dict(title ="Profit"),  
             )
    )   

    fig1 =( 
                    {
                     "data":data_1,
                     "layout":layout_1}
                   )
    
    

    return fig1 , "Date Range {} - {}.".format(str(rangeD[0]),str(rangeD[1])) #, price_chart

# ============================= Price Chart =============================== #
@app.callback(  [
                    Output('subplot_1', 'figure'),
                    Output('subplot_2', 'figure'),
                    Output('subplot_3', 'figure'),
                    Output('subplot_4', 'figure'),
                ],
                [
                    Input('scatter_3d','clickData'), # Date input
                    #Input('scatter_trader','figure'),     
                ],
            ) 

def subplots (clickData):  # just my names (According to order in callback)
    # format: clickData -> datetime
    date = pd.to_datetime(clickData['points'][0]['customdata'])
    
    # window of n historical closing prices
    history_days = 100
    start_date = str(date - timedelta(days=history_days))[:-8]
    # format:  datetime -> str
    end_date = str(date)[:-8]
    
    # Retrieve Technical Analysis data from database
    datasetD =  "ta_series"  # table name
    conn = sql.connect('FX_data/database.db')
    cur = conn.cursor()
    df_ta  = pd.read_sql_query("SELECT * FROM {} WHERE Date>= '{}' AND Date <= '{}'".format(datasetD,start_date,end_date), conn)
    df_ta ['Date'] = pd.to_datetime(df_ta ['Date'])
    df_raw_prices = pd.read_sql_query("SELECT * FROM raw_prices WHERE Date>= '{}' AND Date <= '{}'".format(start_date,end_date), conn)
    df_raw_prices ['Date'] = pd.to_datetime(df_raw_prices ['Date'])    
    conn.close()
 
    # Call functions that generate plotly charts from subplots.py
    subplot_1 = adx(df_ta)
    subplot_2 = candles_bollingerb(df_ta,df_raw_prices)
    subplot_3 = rsi(df_ta)
    subplot_4 = macd(df_ta)

    return subplot_1, subplot_2, subplot_3, subplot_4


if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Feb/2020 12:15:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2020 12:15:26] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2020 12:15:26] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2020 12:15:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2020 12:15:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2020 12:15:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2020 12:16:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2020 12:16:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2020 12:16:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2020 12:16:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2020 12:16:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2020 12:16:42] "POST /_dash-update-component

======================================  END   ===========================================